In [1]:
collections = ['MARKET_INTEL_COLLECTION_US', 'MARKET_INTEL_COLLECTION_EMEA', 'MARKET_INTEL_COLLECTION_LATAM', 'MARKET_INTEL_COLLECTION_APAC']
collection = 'MARKET_INTEL_COLLECTION_US'

In [2]:
service_slugs = ['ubereats-marketintel', 'grubhub-marketintel', 'glovo-marketintel','doordash-marketintel','yelp-marketintel']
service_slug = 'grubhub-marketintel'

In [3]:
begin_date = '2022-06-01'
end_date = '2022-11-03'
days_per_batch = 15

In [4]:
import sys
sys.path.append('/home/maximilian.hofmann/ff_utils/src/')

In [5]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd

In [6]:
import functions as fu
reload(fu)

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_geopandas/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


<module 'functions' from '/home/maximilian.hofmann/tools/brand_science_pricing/data/functions.py'>

In [7]:
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [8]:
with open('/home/maximilian.hofmann/tools/brand_science_pricing/data/queries/sql_ofo_scrape_data_new.sql', 'r') as f:
    sql_template = f.read()

In [9]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_ofo_scrape_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [10]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch)

In [11]:
clustering_fields=['country_code','month']

In [12]:
res = []
with Pool(min(cpu_count(), len(date_list))) as p:
    r = p.starmap(fu.process_ofo_data_date_range, zip(repeat(sql_template), date_list, repeat(collection), repeat(service_slug), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Data cleaning finished!
Start data cleaning...
Data cleaning finished!
Loading formatted geocoded file...
Start data cleaning...
Start data cleaning...
Data cleaning finished!
Start data cleaning...
Start data cleaning...
Data cleaning finished!
Data cleaning finished!
Data cleaning finished!
Start data cleaning...
Data cleaning finished!


KeyError: "['city', 'country_code'] not in index"

In [ ]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_ofo_scrape_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [ ]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [ ]:
upsert_tbl = fu.upsert_tbl_ofo_scrape_data(full_stag_table_name, full_prod_table_name)